In [3]:

vcf_file_address= "22_2k.vcf"  #"22_allps_200.vcf"
vcf_file = open(vcf_file_address,'r');


first_het_variant_s1=True 
blockid_blocks_s1=[]
allele_blocks_s1=[]
varpos_blocks_s1=[]

first_het_variant_s2=True 
blockid_blocks_s2=[]
allele_blocks_s2=[]
varpos_blocks_s2=[]
var_blockids1_blocks_s2=[]   # for each var in s2, shows the s1 blockid


alleles_s5=[]
varposs_s5=[]

header_lines_list=[]
for line in vcf_file:
    line_strip=line.strip()
    if line_strip.startswith('#'):
        header_lines_list.append(line_strip)
        sample_names=line_strip.split('\t')[9:] # last line of header contains sample name
        #print(sample_names)
    else:

        line_parts=line_strip.split('\t')
        #gt_flags, s1, sss, ss, s2, s5 = line_parts[8:]
        gt_flags, s1, s2, s5 = line_parts[8:] # hifi illumina true

        varpos=int(line_parts[1]) # variant position
        gt_flags_split=gt_flags.split(":")
        
        if "PS" in gt_flags_split:
            idx_ps= gt_flags.split(":").index("PS")
            idx_gt= gt_flags.split(":").index("GT")
            
            
            # parental
            split_s5 = s5.split(":")
            blockid_s5 = split_s5[idx_ps]
            allele_s5 = split_s5[idx_gt]
            if allele_s5 !="./.":
                alleles_s5.append(int(allele_s5[0]))
                varposs_s5.append(varpos)

            
            split_s1=s1.split(":")
            blockid_s1 = split_s1[idx_ps]
            allele_s1=split_s1[idx_gt]
            
            if allele_s1=="0/1":allele_s1="./.";blockid_s1="."
                
            if not blockid_s1 == '.':
                if first_het_variant_s1:           # for the first het variant
                    first_het_variant_s1 = False
                    
                    allele_block_s1 = [int(allele_s1[0])]     # alleles of a block
                    varpos_block_s1 = [varpos]               # var pos of a block
                    blockid_blocks_s1.append(int(blockid_s1))   # add the blockid of the first var  to the list of blockids
                    
                else:                              # for the rest of het variants
                    if int(blockid_s1) in blockid_blocks_s1:
                        allele_block_s1.append(int(allele_s1[0]))
                        varpos_block_s1.append(varpos)
                        
                    else: 
                        # add previous block to the list of all blocks
                        allele_blocks_s1.append(allele_block_s1)
                        varpos_blocks_s1.append(varpos_block_s1)

                        # creat new phase block
                        allele_block_s1 = [int(allele_s1[0])]
                        varpos_block_s1 = [varpos]
                        blockid_blocks_s1.append(int(blockid_s1))

                        
            
            split_s2=s2.split(":")
            blockid_s2 = split_s2[idx_ps]
            allele_s2=split_s2[idx_gt]
            
            if allele_s2=="0/1":allele_s2="./.";blockid_s2="."
                
            if not blockid_s2 == '.':
                if first_het_variant_s2:           # for the first het variant
                    first_het_variant_s2 = False
                    
                    allele_block_s2 = [int(allele_s2[0])]     # alleles of a block
                    varpos_block_s2 = [varpos]               # var pos of a block
                    blockid_blocks_s2.append(int(blockid_s2))   # add the blockid of the first var  to the list of blockids
                    var_blockids1_block_s2=[blockid_s1]
                    
                else:                              # for the rest of het variants
                    if int(blockid_s2) in blockid_blocks_s2:
                        allele_block_s2.append(int(allele_s2[0]))
                        varpos_block_s2.append(varpos)
                        var_blockids1_block_s2.append(blockid_s1)
                        
                    else: 
                        # add previous block to the list of all blocks
                        allele_blocks_s2.append(allele_block_s2)
                        varpos_blocks_s2.append(varpos_block_s2)
                        var_blockids1_blocks_s2.append(var_blockids1_block_s2)

                        # creat new phase block
                        allele_block_s2 = [int(allele_s2[0])]
                        varpos_block_s2 = [varpos]
                        blockid_blocks_s2.append(int(blockid_s2))
                        var_blockids1_block_s2=[blockid_s1]
                       
                        
                        
# add last block to the list of all blocks
allele_blocks_s1.append(allele_block_s1)
varpos_blocks_s1.append(varpos_block_s1)

allele_blocks_s2.append(allele_block_s2)
varpos_blocks_s2.append(varpos_block_s2)
var_blockids1_blocks_s2.append(var_blockids1_block_s2)
                        
    
    
number_phaseblocks_s2= len(allele_blocks_s2)
print(number_phaseblocks_s2)

number_phaseblocks_s1= len(allele_blocks_s1)
print(number_phaseblocks_s1)
 
    

        

128
4


In [11]:
#%pwd

for allele_block_s1 in allele_blocks_s1:
    print(len(allele_block_s1))

103
163
668
153


In [12]:
intersected_blocks_idins1_s2=[] # for each block in s2, the corresponding s1 block id

for var_blockids1_blocks_s2_i in var_blockids1_blocks_s2:
    var_blockids1_blocks_s2_i_set=set(var_blockids1_blocks_s2_i)
    
    if '.' in var_blockids1_blocks_s2_i_set:
        var_blockids1_blocks_s2_i_set.discard('.')
        
    if len(var_blockids1_blocks_s2_i_set)==1:
        intersected_idins1_s2=int(list(var_blockids1_blocks_s2_i_set)[0])
    if len(var_blockids1_blocks_s2_i_set)==0:
        intersected_idins1_s2=0
    if len(var_blockids1_blocks_s2_i_set)>1:
        # find the most frequent blockid in the phaseblock
        intersected_idins1_s2=int(max(var_blockids1_blocks_s2_i_set, key=var_blockids1_blocks_s2_i.count) ) 
        
    intersected_blocks_idins1_s2.append(intersected_idins1_s2)


In [13]:
print(blockid_blocks_s1)
#for i in range(len(allele_blocks_s1)):
#    print(len(allele_blocks_s1[i]))
    
#print(blockid_blocks_s2)
#for i in range(len(allele_blocks_s2)):
#    print(len(allele_blocks_s2[i]))

[16050159, 16099924, 16147398, 16324005]


In [14]:
number_phaseblocks_s1= len(allele_blocks_s1)

allele_blocks_s1_new=list(allele_blocks_s1)
for block_i in range(number_phaseblocks_s1): # 
   
    allele_block_s1=allele_blocks_s1[block_i]
    varpos_block_s1=varpos_blocks_s1[block_i]


    similiar_count=0
    diss_count=0
    for var_i, varpos in enumerate(varpos_block_s1):
        allele_s1= allele_block_s1[var_i]
        
        if varpos in varposs_s5:
            allele_s5= alleles_s5[varposs_s5.index(varpos)]
            if allele_s1 == allele_s5:
                similiar_count+=1
            else: 
                diss_count+=1
                
    print(similiar_count, diss_count)          
    if similiar_count < diss_count:
        allele_block_s1_new = [1-i for i in allele_block_s1]
        allele_blocks_s1_new[block_i]=allele_block_s1_new
        


12 7
16 23
39 35
5 24


In [15]:

allele_blocks_s2_new=list(allele_blocks_s2)
for block_i in range(number_phaseblocks_s2): # 
#for block_i, allele_block_s2 in enumerate(allele_blocks_s2):
   
    allele_block_s2=allele_blocks_s2[block_i]
    varpos_block_s2=varpos_blocks_s2[block_i]

    intersected_idins1_s2=intersected_blocks_idins1_s2[block_i]
    if intersected_idins1_s2 !=0:
        block_indx_s1= blockid_blocks_s1.index(intersected_idins1_s2)
    varpos_block_s1=varpos_blocks_s1[block_indx_s1]
    allele_block_s1=allele_blocks_s1_new[block_indx_s1]
    
    intersect_varpos=set(varpos_block_s1) & set(varpos_block_s2)
    similiar_count=0
    for varpos in intersect_varpos:
        allele_s1= allele_block_s1[varpos_block_s1.index(varpos)]
        allele_s2= allele_block_s2[varpos_block_s2.index(varpos)]
        if allele_s1==allele_s2: similiar_count+=1
    #print(similiar_count/len(intersect_varpos))
    if similiar_count < len(intersect_varpos)/2:
        #print("need", similiar_count)
        allele_block_s2_new= [1-i for i in allele_block_s2]
        allele_blocks_s2_new[block_i]=allele_block_s2_new
        


In [16]:
len(blockid_blocks_s1)

4

In [17]:
matrix_address="matrix_up_"+vcf_file_address+".txt"

matrix_file=open(matrix_address,'w');
matrix_file.write("\t\t".join(["Pos","ONT","Illm","parental"])+"\n") # "Pos","ONT","CLR","HiFi","Illm","parental"


for block_i_s1 in range(len(blockid_blocks_s1)):
    blockid_block_s1=blockid_blocks_s1[block_i_s1]
    
    allele_block_s1=allele_blocks_s1_new[block_i_s1]
    varpos_block_s1=varpos_blocks_s1[block_i_s1]
    for var_i_s1 in range(len(varpos_block_s1)):
        varpos=varpos_block_s1[var_i_s1]
        allele_s1=allele_block_s1[var_i_s1]
        
        allele_s2='.'
        blockid_s2='.'
        for block_i_s2 in range(len(blockid_blocks_s2)):
            blockid_block_s2 = blockid_blocks_s2[block_i_s2]
            allele_block_s2 = allele_blocks_s2_new[block_i_s2]
            varpos_block_s2 = varpos_blocks_s2[block_i_s2]
            
            if varpos in varpos_block_s2:
                allele_s2 = allele_block_s2[varpos_block_s2.index(varpos)]
                blockid_s2=blockid_block_s2
                break # when var is found, no need to continue for loop
            
        if varpos in varposs_s5:
            allele_s5 = alleles_s5[varposs_s5.index(varpos)]
        else: allele_s5= '.'

                
        matrix_row_list=[str(varpos),str(allele_s1)+":"+str(blockid_block_s1),
                         str(allele_s2)+":"+str(blockid_s2),str(allele_s5)] 
        matrix_file.write("\t".join(matrix_row_list)+"\n")
    
matrix_file.close()


In [18]:
len(blockid_blocks_s1)

4

In [19]:
varposs_s5[0:10]

[16051453,
 16056353,
 16062995,
 16065078,
 16065536,
 16067208,
 16068295,
 16069374,
 16073196,
 16077252]

In [25]:

for block_i in range(len(allele_blocks_s1_new)): # 
   
    allele_block_s1=allele_blocks_s1_new[block_i]
    varpos_block_s1=varpos_blocks_s1[block_i]


    for var_i, varpos in enumerate(varpos_block_s1):
        allele_s1= allele_block_s1[var_i]
        
        if varpos in varposs_s5:
            allele_s5= alleles_s5[varposs_s5.index(varpos)]
        
            if allele_s1!=allele_s5:
                
                for block_i in range(number_phaseblocks_s2): # 
   
                    allele_block_s2=allele_blocks_s2_new[block_i]
                    varpos_block_s2=varpos_blocks_s2[block_i]
                    if varpos in varpos_block_s2:
                        allele_s2= allele_block_s2[varpos_block_s2.index(varpos)]
                        print(varpos,allele_s1,allele_s2,allele_s5)
                        break

                
                
                
                
                



16069374 0 0 1
16080579 0 0 1
16086302 0 0 1
16087217 0 0 1
16101684 1 1 0
16104385 1 1 0
16105717 1 0 0
16113547 0 0 1
16117006 0 1 1
16120074 1 0 0
16123812 1 0 0
16137152 1 0 0
16142002 1 1 0
16143370 1 1 0
16143995 1 1 0
16144768 1 1 0
16144952 1 1 0
16149963 1 0 0
16159540 0 0 1
16164946 1 1 0
16183780 1 1 0
16183790 1 1 0
16194681 1 0 0
16199059 1 0 0
16201114 1 0 0
16201332 1 0 0
16201360 1 0 0
16203241 1 1 0
16203542 0 1 1
16204790 1 0 0
16204824 1 0 0
16207034 1 0 0
16212603 1 0 0
16213011 1 0 0
16214176 1 1 0
16216383 1 1 0
16217414 1 1 0
16217477 1 1 0
16219445 1 0 0
16223201 1 0 0
16228164 1 1 0
16228270 1 1 0
16230082 1 1 0
16230525 1 1 0
16231828 1 0 0
16240092 1 1 0
16246137 0 0 1
16258516 1 1 0
16267436 1 1 0
16296459 0 0 1
16317765 0 1 1
16335320 1 1 0
16335338 1 1 0
16337995 1 1 0
16360538 1 0 0


In [26]:
for block_i in range(number_phaseblocks_s2): # 
   
    allele_block_s2=allele_blocks_s2_new[block_i]
    varpos_block_s2=varpos_blocks_s2[block_i]

    intersected_idins1_s2=intersected_blocks_idins1_s2[block_i]
    if intersected_idins1_s2 !=0:
        block_indx_s1= blockid_blocks_s1.index(intersected_idins1_s2)
    varpos_block_s1=varpos_blocks_s1[block_indx_s1]
    allele_block_s1=allele_blocks_s1_new[block_indx_s1]
    
    intersect_varpos=set(varpos_block_s1) & set(varpos_block_s2)
    similiar_count=0
    dissimiliar_count=0
    for varpos in intersect_varpos:
        allele_s1= allele_block_s1[varpos_block_s1.index(varpos)]
        allele_s2= allele_block_s2[varpos_block_s2.index(varpos)]
        if allele_s1==allele_s2: 
            similiar_count+=1
        else:
            #print(varpos)
            dissimiliar_count+=1
            
    if similiar_count>1 and dissimiliar_count>1:
        print(similiar_count)

6
3
6
24
9
9
17
6
7
9
26
4
23
10
8
2
6
7
20
12
14
6
15
12
6
6
20
4
9
14
13
10
3
9
2
10
21
4
6


In [22]:
len(allele_block_s2)

11